In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp /content/drive/My\ Drive/FaceMask/face_mask_test.txt /content/
!cp /content/drive/My\ Drive/FaceMask/face_mask_train.txt /content/
!unzip -q /content/drive/My\ Drive/FaceMask/FaceMaskDataset.zip

In [3]:
!git clone https://github.com/qqwweee/keras-yolo3.git

import os
for i in os.listdir("keras-yolo3/"):
  os.system("mv keras-yolo3/"+i+" /content/")

Cloning into 'keras-yolo3'...
remote: Enumerating objects: 144, done.
remote: Total 144 (delta 0), reused 0 (delta 0), pack-reused 144
Receiving objects: 100% (144/144), 151.07 KiB | 626.00 KiB/s, done.
Resolving deltas: 100% (65/65), done.


In [4]:
!pip install keras==2.1.5

%tensorflow_version 1.x
import tensorflow as tf
import sys
import argparse
from yolo import YOLO
from PIL import Image
import os
from yolo3.model import *
from yolo3.utils import *
tf.version.VERSION

     |████████████████████████████████| 337kB 6.4MB/s 
  Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1
TensorFlow 1.x selected.


Using TensorFlow backend.


'1.15.2'

In [0]:
from IPython.display import display

def detecting_image(detector, image):
  punti = []
  plates = []
  if detector.model_image_size != (None, None):
      assert detector.model_image_size[0]%32 == 0, 'Multiples of 32 required'
      assert detector.model_image_size[1]%32 == 0, 'Multiples of 32 required'
      boxed_image = letterbox_image(image, tuple(reversed(detector.model_image_size)))
  else:
      new_image_size = (image.width - (image.width % 32),
                        image.height - (image.height % 32))
      boxed_image = letterbox_image(image, new_image_size)
  image_data = np.array(boxed_image, dtype='float32')

  # print(image_data.shape)
  image_data /= 255.
  image_data = np.expand_dims(image_data, 0)  # Add batch dimension.

  out_boxes, out_scores, out_classes = detector.sess.run(
      [detector.boxes, detector.scores, detector.classes],
      feed_dict={
          detector.yolo_model.input: image_data,
          detector.input_image_shape: [image.size[1], image.size[0]],
          K.learning_phase(): 0
      })

  # print('Found {} boxes for {}'.format(len(out_boxes), 'img'))

  font = ImageFont.truetype(font='font/FiraMono-Medium.otf',
              size=np.floor(3e-2 * image.size[1] + 0.5).astype('int32'))
  thickness = (image.size[0] + image.size[1]) // 300

  for i, c in reversed(list(enumerate(out_classes))):
      predicted_class = detector.class_names[c]
      box = out_boxes[i]
      score = out_scores[i]

      label = '{} {:.2f}'.format(predicted_class, score)
      draw = ImageDraw.Draw(image)
      label_size = draw.textsize(label, font)
      top, left, bottom, right = box

      top = max(0, np.floor(top + 0.5).astype('int32'))
      left = max(0, np.floor(left + 0.5).astype('int32'))
      bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
      right = min(image.size[0], np.floor(right + 0.5).astype('int32'))
      #print(label, (left, top), (right, bottom))

      punti.append([(left,top),(right, bottom)])
      plates.append([left,top,right-left,bottom-top])
  

  return punti,plates,image

In [0]:
f = open("classes.txt","w")
f.write("face\n")
f.write("face_mask")
f.close()

In [0]:
%tensorflow_version 1.x
import cv2
import os
from PIL import Image, ImageFont, ImageDraw
import numpy as np
from matplotlib.colors import rgb_to_hsv, hsv_to_rgb
from IPython.display import display

def bb_intersection_over_union(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

def classification(detector,path_img):

    image = Image.open(path_img)
    w_img,h_img = image.size
    punti, plates, iii = detecting_image(detector,image)

    real_plates = {}
    # for (x, y, w, h) in plates:
    #     real_plates[(x, y, w, h)] = True

    # for (x, y, w, h) in plates:
    #     for (x2, y2, w2, h2) in real_plates.keys():
    #         if real_plates[(x2, y2, w2, h2)] == False:
    #             continue
    #     iou = bb_intersection_over_union([x, y, x + w, y + h], [x2, y2, w2 + x2, h2 + y2])
    #     if (iou > 0.1 and iou != 1.0):
    #         if (w * h) > (w2 * h2):
    #             real_plates[(x2, y2, w2, h2)] = False
    #         else:
    #             real_plates[(x, y, w, h)] = False
      
      
    return punti, real_plates,iii

In [8]:
!git clone https://github.com/swdev1202/keras-yolo3-facedetection.git


Cloning into 'keras-yolo3-facedetection'...
remote: Enumerating objects: 216, done.
remote: Total 216 (delta 0), reused 0 (delta 0), pack-reused 216
Receiving objects: 100% (216/216), 5.75 MiB | 7.84 MiB/s, done.
Resolving deltas: 100% (107/107), done.


In [9]:
from IPython.display import display
from random import randint
from PIL import Image, ImageDraw
import progressbar

test_img = "val/"
model_path = "/content/drive/My Drive/FaceMask/YoloFaceDataAug/ep007-loss18.117-val_loss16.969.h5"


f = open("FACE.txt","w")
f.write("face")
f.close()

arg = {
      "model_path": model_path,
      #"anchors_path": 'model_data/yolo_anchors.txt', #YOLOV3
      "anchors_path": '/content/keras-yolo3-facedetection/model_data/wider_anchors.txt',  #YOLOFACE
      "classes_path": 'classes.txt',
      #"classes_path": 'FACE.txt',
      "score" : 0.3,
      "iou" : 0.45,
      #"model_image_size" : (416, 416), #YOLOV3
      "model_image_size" : (608,608),  #YOLOFACE
      "gpu_num" : 1,
  }
detector = YOLO(**arg)


j = 0
for i in os.listdir(test_img):
  if "jpg" in i :
    img_path = test_img+i
    image = Image.open(img_path)
    iii = detector.detect_image(image)
    iii = image.resize( (int(iii.size[0]/4) , int(iii.size[1]/4)) )
    display(iii)
    #bar.update(j)
    j = j+1
    if j==50:
      break

#bar = progressbar.ProgressBar(max_value=len(os.listdir(test_img))/2)
# j = 0
# for i in os.listdir(test_img):
#   if "jpg" in i :
#     img_path = test_img+i

#     punti, real_plates,iii = classification(detector, img_path)
#     draw = ImageDraw.Draw(iii)
#     for i in punti:
#       draw.rectangle(i)
#     display(iii)
#     print(punti)
#     #bar.update(j)
#     j = j+1
#     if j==10:
#       break

  


Output hidden; open in https://colab.research.google.com to view.

In [10]:
from IPython.display import display
from random import randint
from PIL import Image, ImageDraw
import progressbar

test_img = "val/"
model_path = "/content/drive/My Drive/FaceMask/TinyYoloDataAug/ep029-loss11.815-val_loss11.278.h5"

arg = {
      "model_path": model_path,
      "anchors_path": '/content/model_data/tiny_yolo_anchors.txt', #YOLOV3
      "classes_path": 'classes.txt',
      "score" : 0.3,
      "iou" : 0.45,
      "model_image_size" : (416, 416), #YOLOV3
      "gpu_num" : 1,
  }
detector = YOLO(**arg)


j = 0
for i in os.listdir(test_img):
  if "jpg" in i :
    img_path = test_img+i
    image = Image.open(img_path)
    iii = detector.detect_image(image)
    iii = image.resize( (int(iii.size[0]/4) , int(iii.size[1]/4)) )
    display(iii)
    #bar.update(j)
    j = j+1
    if j==50:
      break

Output hidden; open in https://colab.research.google.com to view.